<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Classification Metrics II

_Authors: Matt Brems, Dave Yerrington, Noelle Brown, Jeff Hale_

---

### Learning Objectives
- Calculate various classification model evaluation metrics.
- Describe the inverse relationship between sensitivity and specificity.
- Understand what the ROC shows and interpret ROC AUC.

In [21]:
# imports
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [3]:
# Read in data.
smoking = pd.read_csv('./data/Whickham.csv')

In [4]:
# Check out first five rows.
smoking.head()

outcome smoker  age
0   Alive    Yes   23
1   Alive    Yes   18
2    Dead    Yes   71
3   Alive     No   67
4   Alive     No   64

- `outcome`: Whether someone is alive or dead.
- `smoker`: Whether somebody smoked or did not smoke.
- `age`: Age in years.

In [7]:
# Turn outcome to 1 if dead and 0 if alive.
smoking['outcome'] = np.where(smoking['outcome']=='Dead', 1, 0)

In [8]:
# Check the distribution of our outcome variable.
smoking['outcome'].value_counts(normalize=True)

0    0.719178
1    0.280822
Name: outcome, dtype: float64

In [13]:
# Turn smoker column to 1 if smoker and 0 if non-smoker.
smoking['smoker'] = smoking['smoker'].map({'Yes': 1, 'No': 0})

In [14]:
smoking.head()

outcome  smoker  age
0        0       1   23
1        0       1   18
2        1       1   71
3        0       0   67
4        0       0   64

One of the interesting things about this data is the relationship between age and smoking.
- You can read more about it [here](https://www2.stat.duke.edu/courses/Spring08/sta103/whickham.pdf).

In [15]:
# How do we usually account for a relationship
# between two independent variables?
smoking['interaction'] = smoking['smoker']*smoking['age']
smoking.head()

outcome  smoker  age  interaction
0        0       1   23           23
1        0       1   18           18
2        1       1   71           71
3        0       0   67            0
4        0       0   64            0

In [16]:
# Set up X and y.
X = smoking.drop(columns = 'outcome')
y = smoking['outcome']

In [18]:
# train/test split.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.25,
                                                    random_state = 42, stratify=y)

In [22]:
# Instantiate model
logreg = LogisticRegression()

In [23]:
# Fit model.
logreg.fit(X_train, y_train)

LogisticRegression()

<details><summary>In this situation, what term would we use to describe someone who is predicted to be dead but actually is alive? (Remember that alive is coded as 0 and dead is coded as 1.)</summary>

- We **falsely** predict someone to be **positive**.
- This would be a **false positive**.
</details>

<details><summary>In this situation, what is a true negative?</summary>

- We **correctly** predict someone to be **negative**.
- Someone who is predicted to be alive (`0`) and actually is alive (`0`).
</details>

<details><summary>If I want to get a good understanding of how our model will do on new data, should I generate a confusion matrix on our training or testing set? Why?</summary>
    
- Testing set!
- If we generate one on our training set, we're going to overestimate the performance of our model... just like if we calculated MSE on our training set.
</details>

In [25]:
# Generate predictions.
preds = logreg.predict(X_test)

In [27]:
# Generate confusion matrix.
# Documentation here: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

cm = confusion_matrix(y_test, preds)
cm

array([[210,  27],
       [ 30,  62]], dtype=int64)

In [28]:
# plot it!
# https://github.com/justmarkham/scikit-learn-tips/blob/master/notebooks/20_plot_confusion_matrix.ipynb

plot_confusion_matrix(logreg, X_test, y_test, cmap='Blues', 
                      values_format='d', display_labels=['alive', 'dead']);

NameError: name 'plot_confusion_matrix' is not defined

### Evaluation Metrics

In [ ]:
# Save TN/FP/FN/TP values.

tn, fp, fn, tp =

# Note that .ravel() will arrange items in a one-dimensional array.
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.ravel.html

**What's the accuracy?**

In [ ]:
# by hand


In [ ]:
# from the model


In [ ]:
# from sklearn metrics


**What's the Specificity?**

In [ ]:
# What is the test specificity of our model?


**What's the sensitivity (recall)?**

In [ ]:
# By hand


In [ ]:
# from sklearn


**What is the precision?**

In [ ]:
# By hand


In [ ]:
# from sklearn


### F1 score

The F1 score is the harmonic mean of precision and recall.

If you care about precision and recall roughly the same amount, F1 score is a great metric to use.

Note that even though all the metrics you’ve seen can be followed by the word score F1 always is. 🤷‍♀️

$$
2*\frac{(\text{Precision}*\text{Recall})} {(\text{Precision} + \text{Recall})}
$$

**What is the F1 Score?**

In [ ]:
# by hand


In [ ]:
# from sklearn


## Relationship between Sensitivity and Specificity

In [ ]:
# Let's create a dataframe called pred_df that contains:
# 1. The true values of our test set.
# 2. The predicted probabilities based on our model.

pred_df = pd.DataFrame({'true_values': y_test,
                        'pred_probs': logreg.predict_proba(X_test)[:,1]})

pred_df.head()

In [ ]:
plt.figure(figsize = (12, 5))

# plot distributions of predicted probabilities by actual values
for group in pred_df.groupby('true_values'):
    sns.distplot(group[1], kde = False, bins = 20, label = f'Actual Outcome = {group[0]}')

plt.xlabel('Predicted Probability that Outcome = 1')
plt.legend();

<details><summary>What do you notice about this distribution?</summary>

- Way more blue than orange.
- There's lots of overlap!
</details>

In [ ]:
plt.figure(figsize = (12, 5))

# plot distributions of predicted probabilities by actual values
for group in pred_df.groupby('true_values'):
    sns.distplot(group[1], kde = False, bins = 20, label = f'Actual Outcome = {group[0]}')

# Add cutoff line


plt.xlabel('Predicted Probability that Outcome = 1')
plt.legend();

<details><summary>Where are my true positives, true negatives, false positives, false negatives in this image?</summary>
    
- True Positive:
    - Items I correctly predict to be positive.
    - The orange bars (actual `1`) that are to the right of the black line (predicted `1`).
- True Negative: 
    - Items I correctly predict to be negative.
    - The blue bars (actual `0`) that are to the left of the black line (predicted `0`).
- False Positive:
    - Items I incorrectly predict to be positive.
    - The blue bars (actual `0`) that are to the right of the black line (predicted `1`).
- False Negative: 
    - Items I incorrectly predict to be negative.
    - The orange bars (actual `1`) that are to the left of the black line (predicted `0`).
</details>

Now, let's adjust our classification threshold to be lower. Instead of using 50% as the cutoff between the positive and negative classes, let's move that threshold down to 10%.
- Any observation with a **predicted probability above 10%** would be **predicted to be in the positive class**.
- Any observation with a **predicted probability below 10%** would be **predicted to be in the negative class**.

<details><summary>When I moved my classification threshold to the left, what happened to sensitivity and specificity?</summary>

- Our number of true negatives decreased and our number of total negatives remains the same.
    - $\text{Specificity} = \frac{TN}{N} \Rightarrow \text{Specificity decreases.}$
- Our number of true positives increased and our number of total positives remains the same.
    - $\text{Sensitivity} = \frac{TP}{P} \Rightarrow \text{Sensitivity increases.}$

</details>

Our goal should be to build a model such that there is **no overlap** between the blue histogram and the orange histogram!
- If there is overlap, we need to recognize the tradeoff between sensitivity and specificity. (As one increases, the other decreases.)
- One measure of how much overlap exists between our distributions is the **area under the ROC curve**.

## Receiver Operating Characteristic (ROC) Curve

Plot the True Positive Rate vs. False Positive Rate for the range of possible decision thresholds and you get the ROC curve!

In [ ]:
# ROC curve

# add worst case scenario line

# add a legend

### Let's check out [this visualization](http://www.navan.name/roc/)

### ROC AUC

The more area under this blue curve is, the better separated our distributions are.
- Check out this gif ([source](https://twitter.com/DrHughHarvey/status/1104435699095404544)):

![](https://media.giphy.com/media/H1SZ5oRLIuZ1t1c4Di/giphy.gif)

We use the **area under the ROC curve** (abbreviated **ROC AUC** or **AUC ROC**) to quantify the gap between our distributions.

<details><summary>Note: Not to be confused with the ROC AOC.</summary>
<img src="./images/AOC.jpg" alt="AOC" width="400"/>
</details>

In [ ]:
# Calculate ROC AUC.


### Interpreting ROC AUC
- If you have an ROC AUC of 0.5, your positive and negative populations perfectly overlap and your model is as bad as it can get.
- If you have an ROC AUC of 1, your positive and negative populations are perfectly separated and your model is as good as it can get.
- The closer your ROC AUC is to 1, the better. (1 is the maximum score.)
- If you have an ROC AUC of below 0.5, your positive and negative distributions have flipped sides. By flipping your predicted values (i.e. flipping predicted 1s and 0s), your ROC AUC will now be above 0.5.
    - Example: You have an ROC AUC of 0.2. If you change your predicted 1s to 0s and your predicted 0s to 1s, your ROC AUC will now be 0.8!

We generate one ROC curve per model. The ROC curve is generated by varying our threshold from 0 to 1. This doesn't actually change the threshold or our original predictions, but it helps us to visualize our tradeoff between _sensitivity_ and _specificity_ and understand how well-separated our populations are.

### Changing your prediction threshold

If you want, you could change your prediction threshold to a custom value.

### Balanced / Unbalanced Classes

Suppose I want to predict the incidence of West Nile Virus (WNV) in Chicago.
- 99.9% of my observations are "no WNV."
- 0.1% of my observations contain "WNV."

If we fit a model and tried to optimize for accuracy, I can predict "no WNV" for every location and have an accuracy score that is really, _really_ good!

<details><summary>Why is this a bad model?</summary>
    
- We'll never predict that a location has West Nile Virus, which is probably going to eventually lead to outbreaks of the disease.
</details>

In classification problems, methods generally perform better when we have roughly equally-sized classes. (i.e. 50% in the positive class and 50% in the negative class for binary classification problems.)

When we do not have each class equally represented, we say we have **unbalanced classes**.

How big of a problem is this? Depends. Realistically, we are probably not going to have a perfect balance of representation from each class. A small amount of imbalance will most likely not matter too much, but a large amount can cause problems. Keep in mind, there are situations where we expect imbalance in our classes (ex. predicting fraud).

#### Methods for Dealing with Unbalanced Classes

- **Weighting observations.** Some models allow the weighting of classes such as linear and logistic regression, Naive Bayes, Random Forests, SVMs, etc. See an example of this [here](https://towardsdatascience.com/weighted-logistic-regression-for-imbalanced-dataset-9a5cd88e68b).
- **Stratified cross-validation.** If we use $k$-fold cross-validation entirely randomly, we may run into issues where some of our folds have no observations from the minority class. Stratifying is almost always a good idea.
- **Change threshold for classification.** By adjusting our classification threshold, we might find a better fit for our particular use-case.
- **Bias correction.** Gary King wrote a [whitepaper](https://gking.harvard.edu/files/gking/files/0s.pdf) on this topic. This is a rigorous approach and while provide good results it's a bit of work.
- **Create synthetic data of minority class.**
- **Oversample minority class.**
- **Undersample majority class.**
- **Combine oversampling majority and undersampling minority classes.**
- **Optimize for a specific metric.**
- **Get more data!** Cop-out answer. But great if you can! 😀

## Interview Questions

<details><summary>What is the ROC curve?</summary>
    
- The ROC curve is a plot of the True Positive Rate (sensitivity) vs. the False Positive Rate (1 - specificity) for all possible decision thresholds.
</details>

<details><summary>Let's say you were building a search engine and wanted to build a classification model that would recommend articles based on the search input. What metric would you want to optimize for and why?</summary>
    
- You could make a case for wanting to minimize false positives (stories that weren't relevant), in which case you'd want to optimize for precision.
- You could make a case for wanting to minimize false negatives (not passing along possibly useful content), in which case you'd want to optimize for recall. 
- Alumni Comment: "The interviewer seemed more interested in seeing if I knew what the metrics were and explaining what priorities would lead me to optimize for one over the other."
</details>

More interview practice questions on these topics [here](https://kiwidamien.github.io/interview-practice-with-precision-and-recall.html)!